In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pickle


# read a file
gdrive_path = '/content/gdrive/My Drive/'


with open(f'{gdrive_path}trainData', 'rb')  as fp:
  data = pickle.load(fp)

In [3]:
import numpy as np
import torch.nn as nn
import torch
from torch.autograd import Variable
from numpy import linalg as LA
from numpy import ma
import os
import pickle
from sklearn.feature_extraction import image
import torch.optim as optim
import torch
import sys
import math

In [4]:
# move data to cuda 
for i in range(len(data)):
  helper = data[i]
  helper[0][0] = [x.to(device='cuda').to(torch.float32) for x in helper[0][0]]
  helper[0][1] = [x.to(device='cuda').to(torch.float32) for x in helper[0][1]] 
  helper[0][2] = [x.to(device='cuda').to(torch.float32) for x in helper[0][2]]
  helper[1] = [x.to(device='cuda').to(torch.float32) for x in helper[1]]
  data[i] = helper

  

In [5]:
# check
data[0][1][2]

tensor([[0.3294, 0.3182, 0.0000,  ..., 0.6981, 0.7055, 0.6477],
        [0.0000, 0.3526, 0.4650,  ..., 0.7217, 0.6978, 0.7010],
        [0.4380, 0.4390, 0.5213,  ..., 0.6960, 0.7132, 0.7188],
        ...,
        [0.6764, 0.6825, 0.7110,  ..., 0.7558, 0.6577, 0.7469],
        [0.6713, 0.6141, 0.6745,  ..., 0.7575, 0.7194, 0.7203],
        [0.6736, 0.6773, 0.7012,  ..., 0.7258, 0.7388, 0.7500]],
       device='cuda:0')

In [6]:
# train functions
def MSEpixelLoss(predictions, y):
    """

    predictions: tensor
        dims = (seqLen, x,y)
    y: list of (x,y)

    return: float
        pixel loss for all images
    """

    #y = torch.stack(y, 0)
    #y = y.to("cuda").to(torch.float32)
    y = y.to(torch.float32)
    loss = torch.nn.MSELoss()(predictions, y)

    return loss

def saveCheckpoint(state, filename):
    torch.save(state, filename)
    print("model checkpoint saved")
    
def loadCheckpoint(checkpoint, model, optimizer):
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    print("loading model complete")
        

def trainLoop(data, model, loadModel, modelName, lr, weightDecay, earlyStopping, criterionFunction, maxIter, epochs, testSet):
    """

    data: list of list of input data and dates and targets
    model: pytorch nn.class
    loadModel: boolean
    modelName: string
        .pth.tar model name on harddrive
    lr: float
    weightDecay: float
    earlyStopping: float
    criterionFunction: nn.lossfunction
    maxIter: int
    epochs: int

    return: nn.class
        trained model
    """
    torch.autograd.set_detect_anomaly(True)
    runningLoss = 0
    stoppingCounter = 0
    lastLoss = 0
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weightDecay)
    
       
    for x in range(epochs):
      # load model 
      if loadModel:
        loadCheckpoint(torch.load(modelName), model = model, optimizer = optimizer)
      model.train()
      for i in range(maxIter):
        # sample data
        idx = np.random.randint(2, size=1)
        helper = data[idx[0]]
        X = helper[0]
        y = helper[1]
        y = torch.stack(y, dim = 0)
        X = [X,y]
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        forward = model.forward(X)
        #predictions = forward[0].to(device='cuda')
        predictions = forward[0]
        loss = MSEpixelLoss(predictions, y) + forward[1]
        loss.backward()
        optimizer.step()

        # print loss
        
        runningLoss += loss.item()
        if i%1000 == 0 and i != 0:
            if testSet != None:
                testLoss = [MSEpixelLoss(model.forward(x[0]).to("cuda"), x[1]).item() for x in testSet]
                testLoss = sum(testLoss)/len(testLoss)
                #idx = np.random.randint(2, size=1)
                #helper = testSet[idx[0]]
                #X = helper[0]
                #y = helper[1]
                #predictions = model.forward(X)
                #testLoss = MSEpixelLoss(predictions, y).item()
                print("current test loss: ", testLoss)
        print("epoch ", x, ", batch ", i, " current loss = ", runningLoss/(i+1))
        
        # early stopping
        if earlyStopping > 0:
            if (lastLoss - loss.item()) < earlyStopping:
                stoppingCounter += 1

            if stoppingCounter == 10:
              print("model converged, early stopping")
              checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
              saveCheckpoint(checkpoint, modelName)
              sys.exit()
        lastLoss = loss.item()
            
      checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
      saveCheckpoint(checkpoint, modelName)
            
    return model

In [7]:
class AE_Transformer(nn.Module):
    def __init__(self, encoderIn, hiddenLenc, hiddenLdec, mlpSize, numLayersDateEncoder, sizeDateEncoder,
                     attLayers, attentionHeads, Training = True, predictionInterval = None):
        super(AE_Transformer, self).__init__()

        # global
        self.training = Training
        self.predictionInterval = predictionInterval

        # encoder
        self.CLayer1 = nn.Conv2d(1, 10, (3, 3), 1)
        self.CLayer2 = nn.Conv2d(10, 20, (3, 3), 1)
        self.CLayer3 = nn.Conv2d(20, 40, (3, 3), 1)
        self.CLayer4 = nn.Conv2d(40, 60, (3, 3), 1)
        self.CLayer5 = nn.Conv2d(60, 80, (3, 3), 1)
        self.CLayer6 = nn.Conv2d(80, 100, (3, 3), 1)
        self.CLayer7 = nn.Conv2d(100, 10, (3, 3), 1)
        self.maxPool = nn.MaxPool2d(3, 1, return_indices=True)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten(start_dim=0, end_dim=-1)

        # date encoder
        self.hiddenLenc = hiddenLenc
        self.sizeDate = sizeDateEncoder
        self.numLayersDateEncoder = numLayersDateEncoder
        In = nn.Linear(3, self.sizeDate)
        Linear = nn.Linear(self.sizeDate, self.sizeDate)
        Out = nn.Linear(self.sizeDate, self.hiddenLenc)
        r = nn.ReLU()
        s = nn.Softmax()
        helper = [In]

        for i in range(self.numLayersDateEncoder):
            helper.append(Linear)
            helper.append(r)
        helper.append(Out)
        helper.append(s)
        self.dateEncoderMLPlist = nn.ModuleList(helper)


        # MLP Encoder
        self.encoderIn = encoderIn
        In = nn.Linear(self.encoderIn, self.hiddenLenc)
        Linear = nn.Linear(self.hiddenLenc, self.hiddenLenc)
        out = nn.Linear(self.hiddenLenc, self.hiddenLenc)
        r = nn.ReLU()
        helper = [In]

        for i in range(mlpSize):
            helper.append(Linear)
            helper.append(r)
        helper.append(out)
        self.MLPlistEnc = nn.ModuleList(helper)

        # hidden space
        self.attentionLayers = attLayers
        self.attentionHeads = attentionHeads
        self.transformer = nn.Transformer(d_model = self.hiddenLenc, nhead= self.attentionHeads, num_encoder_layers=self.attentionLayers, num_decoder_layers=self.attentionLayers)

        # MLP Decoder
        self.hiddenLdec = hiddenLdec
        In = nn.Linear(self.hiddenLdec, self.hiddenLdec)
        Linear = nn.Linear(self.hiddenLdec, self.hiddenLdec)
        out = nn.Linear(self.hiddenLdec, self.hiddenLdec)
        r = nn.ReLU()
        helper = [In]

        for i in range(mlpSize):
            helper.append(Linear)
            helper.append(r)
        helper.append(out)
        self.MLPlistDec = nn.ModuleList(helper)

        # decoder
        self.TCLayer1 = nn.ConvTranspose2d(10, 100, (3, 3), 1)
        self.TCLayer2 = nn.ConvTranspose2d(100, 80, (3, 3), 1)
        self.TCLayer3 = nn.ConvTranspose2d(80, 60, (3, 3), 1)
        self.TCLayer4 = nn.ConvTranspose2d(60, 40, (3, 3), 1)
        self.TCLayer5 = nn.ConvTranspose2d(40, 20, (3, 3), 1)
        self.TCLayer6 = nn.ConvTranspose2d(20, 10, (3, 3), 1)
        self.TCLayer7 = nn.ConvTranspose2d(10, 1, (3, 3), 1)
        self.maxUnPool = nn.MaxUnpool2d(3, 1)

    def dateEncoder(self, dateVec):
        """
        date encoder uses dates as input and projects onto single scalar number with sigmoid activation in last layer
        dateVec: tensor
            input date as vector [day, month, year]
        return: float
        """

        # MLP
        s = dateVec.to("cuda")
        for layer in self.dateEncoderMLPlist:
            s = layer(s)
        return s

    def encoder(self, x,  encDate):
        """
        downscale channel dimensions then flatten and weight with output of date Encoder

        x: tensor
            Input image patches (5,50,50)
        encDate: 1d tensor
            len = output encoder for each input image
        return: list of tensor, list of tensor and list of tensor
            [flattened image vectors for latent space, skip connections list for each image in input,
            list indices for 2dmaxunpool in decoder]
        """
        # init memory; idea: safe maxPool indices and skip connections in list of lists for decoder
        result = torch.zeros((len(x), self.hiddenLenc)).to("cuda")
        poolingIndices = []
        skipConnections = []

        for i in range(len(x)):  # add max pooling
            helper = []
            helper1 = []
            image = x[i].to("cuda")
            image = image.view(1,50,50)

            # start convolutions
            s = self.CLayer1(image)
            s, indices = self.maxPool(s)
            s = self.relu(s)
            helper.append(s)
            helper1.append(indices)

            s = self.CLayer2(s)
            s, indices = self.maxPool(s)
            s = self.relu(s)
            helper.append(s)
            helper1.append(indices)

            s = self.CLayer3(s)
            s, indices = self.maxPool(s)
            s = self.relu(s)
            helper.append(s)
            helper1.append(indices)

            s = self.CLayer4(s)
            s, indices = self.maxPool(s)
            s = self.relu(s)
            helper.append(s)
            helper1.append(indices)

            s = self.CLayer5(s)
            s, indices = self.maxPool(s)
            s = self.relu(s)
            helper.append(s)
            helper1.append(indices)

            s = self.CLayer6(s)
            s, indices = self.maxPool(s)
            s = self.relu(s)
            helper.append(s)
            helper1.append(indices)

            s = self.CLayer7(s)
            s, indices = self.maxPool(s)
            s = self.relu(s)

            helper.append(s)
            helper1.append(indices)
            s = self.flatten(s)

            # MLP
            for layer in self.MLPlistEnc:
                s = layer(s)
            result[i, :] = s + self.dateEncoder(encDate[i]) # + encoder temporal embedding

            # save lists for decoder
            skipConnections.append(helper)
            poolingIndices.append(helper1)

        return [result, skipConnections, poolingIndices]

    def positionalEncodings(self, seqLen):
        """
        creates positional encoding matrix for the transformer model based on vasmari et al

        seqLen: int
            length of sequence
        inputLen:
            length of input

        returns: 2d tensor
            constants added for positional encodings
        """
        # create constant 'pe' matrix with values dependant on pos and i
        pe = torch.zeros(seqLen, self.hiddenLenc).to("cuda")
        for pos in range(seqLen):
            for i in range(0, self.hiddenLenc, 2):
                pe[pos, i] = \
                    math.sin(pos / (10000 ** ((2 * i) / self.hiddenLenc)))
                pe[pos, i + 1] = \
                    math.cos(pos / (10000 ** ((2 * (i + 1)) / self.hiddenLenc)))

        pe = pe.unsqueeze(0)

        # make relatively larger
        x = pe * math.sqrt(self.hiddenLenc)
        pe = pe + x

        return pe

    def get_tgt_mask(self, size):
        """
        Generates target mask for decoder in nn.Transformer

        size: int
            size of embedded images

        returns tensor
            size =  (size,size)
        """
        mask = torch.tril(torch.ones(size, size) == 1).to("cuda")
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, float(0.0))

        return mask



    def latentSpace(self, flattenedInput, targets, targetsT):
        """
        gets flattened feature vectors from encoder and feeds them into transformer

        flattenedInput: tensor
            output of encoder
        targets: tensor
            if training == True
        targetsT: list of tensor
            temporal information of images

        returns: tensor
            output from the transformer, same shape as input

        """
        if self.training: # ~teacher forcing
            # add start token to sequences
            helper = torch.zeros(1, self.hiddenLenc, dtype=torch.float32).to("cuda")
            flattenedInput = torch.vstack([helper, flattenedInput]).to("cuda")
            targets = self.encoder(targets, targetsT)[0].to("cuda") # add temporal information
            targetsOut = targets.clone().to("cuda") # for latentspace loss
            targets = torch.vstack([helper, targets]).to("cuda")

            # positional information to input
            positionalEmbedding = self.positionalEncodings(flattenedInput.size(0) * 2).to("cuda")

            # divide for input and output
            idx = int(flattenedInput.size(0))
            inputMatrix = positionalEmbedding[:, 0:idx, :]
            targetMatrix = positionalEmbedding[:, idx:flattenedInput.size(0)*2, :]

            flattenedInput = flattenedInput + inputMatrix
            flattenedInput = flattenedInput.squeeze(0)

            # add positional information
            targets = targets + targetMatrix
            targets = targets.squeeze(0)

            targetMask = self.get_tgt_mask(targets.size(0))
            out = self.transformer(flattenedInput, targets, tgt_mask = targetMask)
            out = out[1:, :]

            # MSE loss for latent space
            loss = nn.MSELoss()(out, targetsOut)

            return [out, loss]

        if self.training == False: # inference
            ### len(targetsT) = self.predictionInterval -> variable prediction length
            # add start token to sequences
            yInput = torch.zeros(1, self.hiddenLenc, dtype=torch.float32)
            helper = torch.zeros(1, self.hiddenLenc, dtype=torch.float32)
            flattenedInput = torch.vstack([helper, flattenedInput])

            for q in range(self.predictionInterval):
                # positional information to input
                positionalEmbedding = self.positionalEncodings(flattenedInput.size(0) + (q+1))
                positionalEmbedding = positionalEmbedding.squeeze()
                flattenedInput = flattenedInput + positionalEmbedding[0:flattenedInput.size(0)]

                yInput = yInput + positionalEmbedding[flattenedInput.size(0):]

                # get mask
                targetMask = self.get_tgt_mask(yInput.size(0))

                # forward pass
                out = self.transformer(flattenedInput, yInput, tgt_mask=targetMask)
                out = out + self.dateEncoder(targetsT[q]) # add temporal information
                nextItem = out[-1]
                yInput = torch.vstack([yInput, nextItem]).squeeze()

            return yInput[1:, :]


    def decoder(self, latentOutput, skips, indices):
        """
        transposed convolutions to get the original image shape

        latentOutput: tensor
            output of latent space
        decdate: 1d tensor
            dates for predictions
        skips: list of tensor
            skip connections
        indices: list of tensor
            indices for maxunpool in recosntruction of the image
        return: tensor
            output NDSI image snow maks of shape (5, 50,50) # 5 timepoints
        """

        result = torch.zeros((latentOutput.size(0), 50, 50)).to("cuda")
        for i in range(latentOutput.size(0)):
            image = latentOutput[i, :].clone()

            # MLP
            s = image.clone().to("cuda")
            for layer in self.MLPlistDec:
                s = layer(s)

            # start deconvolution; image should be (100, 8, 8)
            image = torch.reshape(s, (10, 22, 22))

            s = image + skips[i][6]
            s = self.maxUnPool(image, indices[i][6])
            s = self.TCLayer1(s)
            s = self.relu(s)

            s = s + skips[i][5]
            s = self.maxUnPool(s, indices[i][5])
            s = self.TCLayer2(s)
            s = self.relu(s)

            s = s + skips[i][4]
            s = self.maxUnPool(s, indices[i][4])
            s = self.TCLayer3(s)
            s = self.relu(s)

            s = s + skips[i][3]
            s = self.maxUnPool(s, indices[i][3])
            s = self.TCLayer4(s)
            s = self.relu(s)

            s = s + skips[i][2]
            s = self.maxUnPool(s, indices[i][2])
            s = self.TCLayer5(s)
            s = self.relu(s)

            s = s + skips[i][1]
            s = self.maxUnPool(s, indices[i][1])
            s = self.TCLayer6(s)
            s = self.relu(s)

            s = s + skips[i][0]
            s = self.maxUnPool(s, indices[i][0])
            s = self.TCLayer7(s)
            s = self.relu(s)

            # save in tensor
            result[i, :, :] = s

        return result

    def forward(self, d):
        """
        forward pass
        d: list of tensor and encoder and decoder date vectors
            input data
        returns: list of tensor and int
            if training: model predictions and latent space loss
            else: model predictions

        """

        # get data
        inpt = torch.stack(d[0][0], dim = 0).to("cuda")
        s = inpt[:, 9, :, :].float().to("cuda")
        datesEncoder = d[0][1]
        datesDecoder = d[0][2]
        target = d[1].float().to("cuda")

        # encoder
        res = self.encoder(s, datesEncoder)

        # latent space flattenedInput, targets, targetsT, positionalEmbedding
        l = self.latentSpace(res[0], target, datesDecoder)

        if self.training:
            # decoder
            s = self.decoder(l[0], res[1], res[2]) # output encoder: [result, skipConnections, poolingIndices]

            return [s, l[1]]
        elif self.training == False:
            # decoder
            s = self.decoder(l, res[1], res[2])  # output encoder: [result, skipConnections, poolingIndices]

            return s



In [ ]:
crit = 0.90
dTrain = data[0:round(len(data)*crit)]
dTest = data[round(len(data)*crit):-1]

# model
#model = models.VAE_LSTM(484, 10, 10, 10, 10, 484, 484)
#model = VAE_LSTM(484, 10, 10, 10, 10, 484, 484) # works reasonably good. loss 0.04 
#model = models.VAE_LSTM(484, 10, 5, 5, 10, 484, 484) # not as good
model = AE_Transformer(4840,4840,4840, 3, 1, 1000, 2, 2, True, None)
model = model.to("cuda").to(torch.float32)
#model


res = trainLoop(dTrain, model, False,f"{gdrive_path}aAETransformer.pth.tar", 0.001, 0.01, -1, MSEpixelLoss, 5000, 1, None)

#hiddenL, mlpSize, numLayersDateEncoder, sizeDateEncoder, lstmLayers, lstmHiddenSize, lstmInputSize


<ipython-input-7-39828ed8d21f>:95: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s = layer(s)


epoch  0 , batch  0  current loss =  1.1162177324295044
epoch  0 , batch  1  current loss =  1.1372985243797302
epoch  0 , batch  2  current loss =  1.1387696663538616
epoch  0 , batch  3  current loss =  1.121318906545639
epoch  0 , batch  4  current loss =  1.109349536895752
epoch  0 , batch  5  current loss =  1.1000985503196716
epoch  0 , batch  6  current loss =  1.099532621247428
epoch  0 , batch  7  current loss =  1.0920673310756683
epoch  0 , batch  8  current loss =  1.0853986740112305
epoch  0 , batch  9  current loss =  1.078763234615326
epoch  0 , batch  10  current loss =  1.0715147961269726
epoch  0 , batch  11  current loss =  1.0619187504053116
epoch  0 , batch  12  current loss =  1.0431132729236896
epoch  0 , batch  13  current loss =  0.9861746621983392
epoch  0 , batch  14  current loss =  2.73040766119957
epoch  0 , batch  15  current loss =  2.6001441348344088
epoch  0 , batch  16  current loss =  2.4942526098559883
epoch  0 , batch  17  current loss =  2.4122423